# Remove before submtion for our refrenc only
## Yusuf update 
I completed the code for part one, but there's a huge error in the calculation of the inverse kinematics. I increased the max number of function evaluations to 5000 to give the optimizer more chances to find a solution, but the errors are still quite large.

## The Three Transformation Matrices

T_base:
- Transforms from the "world" coordinate system to our robot's base coordinate system.
- Accounts for where our robot is positioned/oriented in the world.

T_final:
- The transformation we calculate using D‑H parameters.
- Goes from the robot's base frame to the end‑effector frame.
- This is the forward kinematics computes we did in lab 2 part 4.

T_tool:
- Transforms from the end‑effector frame to the actual tool frame.
- Accounts for any gripper attached to the end‑effector.

Overall Transformation:
- The overall transformation from the world frame to the tool frame is given by:
    $$T_{\text{total}} = T_{\text{base}} \; T_{\text{final}} \; T_{\text{tool}}$$
- Since our world frame is the same as the robot base frame then T_base is the identity matrix. 
- So, if we have a tool attached, the overall transformation simplifies to:
  $$T_{\text{total}} = T_{\text{final}} \; T_{\text{tool}}$$
- If no tool is attached, T_tool is also the identity matrix, and $$T_{\text{total}} = T_{\text{final}}$$


In [1]:
import math
import numpy as np
import pandas as pd
from sympy.matrices import Matrix
from scipy.optimize import least_squares
from sympy  import symbols, cos, sin, atan2, pi, lambdify 
from pathlib import Path

In [ ]:
#Global variables
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2
# Workspace safety limits (mm)
MIN_REACH = 50.0      # Minimum distance from base
MAX_Z = 400.0         # Maximum height
MIN_Z = -50.0         # Minimum height (safety margin above ground)
# Joint angle limits (degrees) - will be updated from robot if available
JOINT_LIMITS = [
    (-165, 165),  # J1
    (-90, 90),    # J2
    (-180, 65),   # J3
    (-160, 160),  # J4
    (-115, 115),  # J5
    (-175, 175)   # J6
]

- Recursively searches the current working directory (Path.cwd()) for files named `mecharm_control_group_5.csv`.
- Selects the first match whose parent folder path contains the substring `Lab3`.
- If no suitable file is found it raises a FileNotFoundError with instructions on where to place the CSV or how to change the notebook working directory.
- If found, it prints the file path and loads the CSV into a pandas DataFrame named `data`.
- `known_joint_angles` is a NumPy array extracted from the DataFrame containing joint angles J1 to J6.
- `known_position` is a NumPy array extracted from the DataFrame containing end-effector positions X, Y, Z.
- `known_orientation` is a NumPy array extracted from the DataFrame containing end-effector orientations RX, RY, RZ.

In [3]:
for p in Path.cwd().rglob('mecharm_control_group_5.csv'):
    if 'Lab3' in str(p.parent):
        csv_path = p
        break

if csv_path.exists() == False:
    raise FileNotFoundError("mecharm_control_group_5.csv not found. Put it in Lab3/ under the repo or set notebook cwd to the repo root (use %cd).")
else:
    print("Loading CSV from:", csv_path)
    data = pd.read_csv(csv_path)
    
#Joint Angles from Lab 2 task 3
known_joint_angles = data[['J1','J2','J3','J4','J5','J6']].to_numpy()
#End-Effector Positions from Lab 2 task 3
known_position = data[['X','Y','Z']].to_numpy()
#End-Effector Orientations from Lab 2 task 3
known_orientation = data[['RX','RY','RZ']].to_numpy()

Loading CSV from: /Users/yusufdineh/Documents/GitHub/EE347/Lab3/mecharm_control_group_5.csv


- we are defining our DH table parameters here in a 6x4 matrix called `dh_table`
- DH table rows: [a, alpha, d, offset, offset]

In [ ]:
dh_table = [
    [0.049,     0,          135.926,    q1],           # joint 1
    [0,     -right,     0,      q2 - right],   # joint 2
    [99.973,   0,          0,      q3],           # joint 3
    [10.012,    -right,     107.011,     q4],           # joint 4
    [0.003,     right,      0.006,      q5],           # joint 5
    [0.054,     -right,     64.973,   q6]              # joint 6
]


# dh_table = [
#     [0,     0,          125.91,    q1],           # joint 1
#     [0,     -right,     0,      q2 - right],   # joint 2
#     [112.06,   0,          0,      q3],           # joint 3
#     [15,    -right,     97,     q4],           # joint 4
#     [0,     right,      0,      q5],           # joint 5
#     [0,     -right,     67,   q6]              # joint 6
# ]

- `get_transformation_matrix(a, alpha, d, theta)`
  - Inputs: the four DH parameters for one joint:
  - Returns: a 4×4 SymPy homogeneous transform Matrix that encodes the rotation and translation from frame i to frame i+1 for that DH row.

- `overall_transformation(dh_table)`
  - `T = Matrix(np.identity(4))` initializes the cumulative transform as the 4×4 identity (base frame).  
  - Loop: for each DH row `a, alpha, d, theta` it calls `get_transformation_matrix` to get the per‑joint transform `T_i`, then multiplies `T = T * T_i`. That sequential multiplication composes transforms frame‑by‑frame, so after the loop `T` is the full symbolic transform from the robot base to the end‑effector.  
  - Return: the composed 4×4 SymPy Matrix (use `T[:3,3]` for position and `T[:3,:3]` for rotation when lambdifying).

In [5]:
def get_transformation_matrix(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), cos(alpha)*d],
        [0, 0, 0, 1]
    ])
def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, theta)
        T = T * T_i
    return T

- `T_symbolic` Builds a single 4×4 SymPy homogeneous transform from base → end‑effector using your DH table. Result is symbolic (contains q1..q6).
- `fk_num` Create a fast numeric function that maps numeric (q1..q6) → position vector (x,y,z).
- `rot_num` = Create a fast numeric function that maps numeric (q1..q6) → 3×3 rotation matrix.

In [6]:
T_symbolic = overall_transformation(dh_table)
fk_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, 3], modules='numpy')
rot_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, :3], modules='numpy')

`fk` computes end‑effector position [x,y,z] from joint angles using the lambdified forward‑kinematics function fk_num and returns it as a NumPy array.

In [7]:
def fk(q_vals):
    q = np.asarray(q_vals, dtype=float)
    q = np.radians(q)
    xyz = np.asarray(fk_num(*q), dtype=float).ravel()
    return xyz


# Inverse kinematics helper functions

- `position_error`: compute the error between the forward-kinematics position and the desired target.
- `orientation_error`: compute rotational error (ZYX yaw-pitch-roll) between current forward-kinematics rotation and desired orientation.
  - Helper `_wrap_angle(a)` wraps any angle to [-π, π].
- `link_lengths` computea a upper bound on robot reach from DH rows.

In [8]:
def rotation_matrix_to_euler_zyx(R, transpose=False):
    """
    Extract Z-Y-X Euler angles (yaw, pitch, roll) from a 3x3 rotation matrix.
    
    The matrix convention is:
    R = [[cos(γ)cos(β),  -cos(α)sin(γ) + cos(γ)sin(α)sin(β),   sin(γ)sin(α) + cos(γ)cos(α)sin(β)],
         [cos(β)sin(γ),   cos(γ)cos(α) + sin(γ)sin(α)sin(β),  -cos(γ)sin(α) + cos(α)sin(γ)sin(β)],
         [-sin(β),        cos(β)sin(α),                         cos(α)cos(β)]]
    
    Where: γ = yaw, β = pitch, α = roll
    
    Args:
        R: 3x3 rotation matrix
        transpose: If True, use R.T instead of R (default False)
        
    Returns:
        (roll, pitch, yaw) in radians
    """
    # Apply transpose if needed
    if transpose:
        R = R.T
    
    # Extract pitch from R[2,0] = -sin(β)
    sin_beta = -R[2, 0]
    sin_beta = np.clip(sin_beta, -1.0, 1.0)  # Clamp for numerical stability
    pitch = np.arcsin(sin_beta)
    
    cos_beta = np.cos(pitch)
    
    # Check for gimbal lock (cos(β) ≈ 0, i.e., pitch ≈ ±90°)
    if np.abs(cos_beta) < 1e-6:
        # Gimbal lock: set yaw = 0 and extract roll from remaining matrix elements
        yaw = 0.0
        roll = np.arctan2(-R[0, 1], R[1, 1])
    else:
        # Normal case: extract roll and yaw from matrix elements
        # From R[2,1] = cos(β)sin(α)  and  R[2,2] = cos(α)cos(β)
        roll = np.arctan2(R[2, 1] / cos_beta, R[2, 2] / cos_beta)
        
        # From R[0,0] = cos(γ)cos(β)  and  R[1,0] = cos(β)sin(γ)
        yaw = np.arctan2(R[1, 0] / cos_beta, R[0, 0] / cos_beta)
    
    return roll, pitch, yaw

In [9]:
def position_error(q_position, x_target, y_target, z_target):
    """
    Compute position error between FK result and target.
    
    Args:
        q_position: Joint angles in degrees (6-element array)
        x_target, y_target, z_target: Target position in mm
        
    Returns:
        3-element error vector in mm
    """
    pos = fk(q_position)  # fk() handles degree→radian conversion internally
    return np.array([pos[0] - float(x_target),
                     pos[1] - float(y_target),
                     pos[2] - float(z_target)])

def orientation_error(q_orientation, rx_d, ry_d, rz_d):
    def _wrap_angle(a):
        return (a + np.pi) % (2*np.pi) - np.pi

    q = np.asarray(q_orientation, dtype=float)
    q_rad = np.radians(q)
    rx_d_rad = np.radians(rx_d)
    ry_d_rad = np.radians(ry_d)
    rz_d_rad = np.radians(rz_d)

    R = np.array(rot_num(*q_rad), dtype=float)
    # Remove transpose=True - use the matrix directly
    roll, pitch, yaw = rotation_matrix_to_euler_zyx(R, transpose=False)

    err_roll  = _wrap_angle(roll  - rx_d_rad)
    err_pitch = _wrap_angle(pitch - ry_d_rad)
    err_yaw   = _wrap_angle(yaw   - rz_d_rad)

    return np.array([err_roll, err_pitch, err_yaw])

def link_lengths(dh_table):
    """Compute upper bound on robot reach from DH parameters."""
    link_lengths = 0.0
    for a, alpha, d, theta in dh_table:
        link_lengths += abs(float(a)) + abs(float(d))
    return link_lengths

- `inverse_kinematics` computes joint angles [q1..q6] to reach desired end-effector pose using numerical optimization.

In [10]:
def validate_workspace(x: float, y: float, z: float) -> None:
    """
    Validate target position is within safe workspace.
    
    Args:
        x, y, z: Target position in mm
        
    Raises:
        ValueError: If position is outside safe workspace
    """
    distance = math.sqrt(x*x + y*y + z*z)
    if distance < MIN_REACH:
        raise ValueError(f"Target too close to base: {distance:.1f}mm < {MIN_REACH}mm")
    
    if z > MAX_Z:
        raise ValueError(f"Target too high: {z:.1f}mm > {MAX_Z}mm")
    
    if z < MIN_Z:
        raise ValueError(f"Target too low: {z:.1f}mm < {MIN_Z}mm")
    
def inverse_kinematics(x_target: float, y_target: float, z_target: float, 
                       rx_d: float, ry_d: float, rz_d: float, 
                       q_init: np.ndarray, max_reach: float,
                       max_iterations: int = 5000, tolerance: float = 1e-6) -> np.ndarray:
    """
    Compute inverse kinematics using numerical optimization.
    
    Simultaneously optimizes position and orientation to find joint angles
    that achieve the target pose.
    
    Args:
        x_target, y_target, z_target: Target position in mm
        rx_d, ry_d, rz_d: Target orientation (roll, pitch, yaw) in degrees
        q_init: Initial guess for joint angles in degrees
        max_reach: Maximum robot reach in mm
        max_iterations: Maximum optimization iterations
        tolerance: Convergence tolerance
        
    Returns:
        Array of 6 joint angles in degrees
        
    Raises:
        ValueError: If target is unreachable or IK fails to converge
    """
    # Validate target is within workspace
    validate_workspace(x_target, y_target, z_target)
    
    distance_from_origin = math.hypot(x_target, y_target, z_target)
    if distance_from_origin > max_reach:
        raise ValueError(
            f"Target outside robot reach: {distance_from_origin:.1f}mm > {max_reach:.1f}mm"
        )
    
    q_init = np.asarray(q_init, dtype=float)
    
    def combined_residual(q_all, pos_weight=1.0, ori_weight=10.0):
        """
        Compute combined position and orientation residual.
        
        Returns 6-element vector: [pos_x, pos_y, pos_z, ori_roll, ori_pitch, ori_yaw]
        """
        pos_err = position_error(q_all, x_target, y_target, z_target)
        ori_err = orientation_error(q_all, rx_d, ry_d, rz_d)
        return np.concatenate([pos_err * pos_weight, ori_err * ori_weight])
    
    # Extract bounds from JOINT_LIMITS
    lower_bounds = np.array([limit[0] for limit in JOINT_LIMITS])
    upper_bounds = np.array([limit[1] for limit in JOINT_LIMITS])
    bounds = (lower_bounds, upper_bounds)
    
    # Use trust region reflective method for better non-linear handling with bounds
    result = least_squares(
        combined_residual, 
        q_init,
        bounds=bounds,  # Apply joint angle limits as bounds
        method='trf',   # 'trf' supports bounds
        max_nfev=max_iterations,
        ftol=tolerance,
        xtol=tolerance,
        verbose=0
    )
    
    if not result.success:
        print(f"IK optimization warning: {result.message}")
    
    q_solution = result.x
    
    # Verify solution quality
    final_pos_error = np.linalg.norm(position_error(q_solution, x_target, y_target, z_target))
    final_ori_error = np.linalg.norm(orientation_error(q_solution, rx_d, ry_d, rz_d))
    
    # More reasonable thresholds (in degrees for orientation)
    if final_pos_error > 10.0:  # 10mm position error
        print(f"Warning: Large position error: {final_pos_error:.2f} mm")
    
    if final_ori_error > np.radians(10):  # 10° orientation error
        print(f"Warning: Large orientation error: {np.degrees(final_ori_error):.2f}°")
    
    return q_solution

## Validation and Accuracy Check

In [11]:
# %% [markdown]
# # Enhanced Inverse Kinematics Testing Suite - HONEST VERSION
# This version properly tests your IK by NOT giving it easy initial guesses.
# It will expose real problems with your inverse kinematics algorithm.



def angle_diff_deg(a_deg, b_deg):
    """Compute signed angle difference in degrees, wrapped to [-180, 180]."""
    d = (a_deg - b_deg + 180.0) % 360.0 - 180.0
    return d

def test_single_pose(target_pos, target_ori, known_joints, q_init, 
                     max_iterations=5000, tolerance=1e-6, test_index=None):
    """
    Test IK for a single target pose and compute comprehensive error metrics.
    
    Args:
        target_pos: [x, y, z] target position in mm
        target_ori: [rx, ry, rz] target orientation in degrees
        known_joints: Known correct joint angles in degrees (ground truth)
        q_init: Initial guess for joint angles in degrees
        max_iterations: Max optimizer iterations
        tolerance: Convergence tolerance
        test_index: Optional index for this test case
        
    Returns:
        dict: Comprehensive test results including all error metrics
    """
    x, y, z = target_pos
    rx, ry, rz = target_ori
    
    # Use your link_lengths function to compute reach
    reach = link_lengths(dh_table)
    
    # Initialize result dictionary
    result = {
        'index': test_index,
        'target_x': x,
        'target_y': y,
        'target_z': z,
        'target_rx': rx,
        'target_ry': ry,
        'target_rz': rz,
    }
    
    # Store known (ground truth) joint angles
    for i, angle in enumerate(known_joints, 1):
        result[f'known_j{i}'] = angle
    
    # Store initial guess used
    for i, angle in enumerate(q_init, 1):
        result[f'init_j{i}'] = angle
    
    # Attempt inverse kinematics using YOUR function
    try:
        solved_joints = inverse_kinematics(x, y, z, rx, ry, rz, q_init, reach,
                                         max_iterations=max_iterations, tolerance=tolerance)
        solved_joints = np.asarray(solved_joints, dtype=float)
        
        # Store solved joint angles
        for i, angle in enumerate(solved_joints, 1):
            result[f'solved_j{i}'] = angle
        
        # Compute per-joint errors
        joint_errors_signed = np.array([angle_diff_deg(solved_joints[i], known_joints[i]) 
                                       for i in range(6)])
        joint_errors_abs = np.abs(joint_errors_signed)
        
        for i, (signed, absolute) in enumerate(zip(joint_errors_signed, joint_errors_abs), 1):
            result[f'j{i}_error_signed'] = signed
            result[f'j{i}_error_abs'] = absolute
        
        # Joint space error metrics
        result['joint_error_mean'] = np.mean(joint_errors_abs)
        result['joint_error_max'] = np.max(joint_errors_abs)
        result['joint_error_rms'] = np.sqrt(np.mean(joint_errors_abs**2))
        result['joint_error_total'] = np.sum(joint_errors_abs)
        
        # CRITICAL: Use solved joints with YOUR FK function to see actual end-effector error
        actual_pos = fk(solved_joints)
        
        # Task space position errors
        pos_error_vec = actual_pos - target_pos
        result['fk_pos_error_x'] = pos_error_vec[0]
        result['fk_pos_error_y'] = pos_error_vec[1]
        result['fk_pos_error_z'] = pos_error_vec[2]
        result['fk_pos_error_norm'] = np.linalg.norm(pos_error_vec)
        
        # Also compute position error if we used known joints (should be ~0)
        known_pos = fk(known_joints)
        known_pos_error = np.linalg.norm(known_pos - target_pos)
        result['known_fk_pos_error'] = known_pos_error
        
        # Success indicators
        result['converged'] = True
        result['status'] = 'success'
        
        # Quality assessment based on TASK SPACE error (what matters!)
        if result['fk_pos_error_norm'] < 1.0:
            result['quality'] = 'excellent'
        elif result['fk_pos_error_norm'] < 5.0:
            result['quality'] = 'good'
        elif result['fk_pos_error_norm'] < 10.0:
            result['quality'] = 'acceptable'
        elif result['fk_pos_error_norm'] < 50.0:
            result['quality'] = 'poor'
        else:
            result['quality'] = 'failed'
            
    except Exception as e:
        # IK failed
        result['converged'] = False
        result['status'] = f'failed: {str(e)}'
        result['quality'] = 'failed'
        
        # Fill with NaN for missing values
        for i in range(1, 7):
            result[f'solved_j{i}'] = np.nan
            result[f'j{i}_error_signed'] = np.nan
            result[f'j{i}_error_abs'] = np.nan
        
        for key in ['joint_error_mean', 'joint_error_max', 'joint_error_rms', 'joint_error_total',
                   'fk_pos_error_x', 'fk_pos_error_y', 'fk_pos_error_z', 'fk_pos_error_norm',
                   'known_fk_pos_error']:
            result[key] = np.nan
    
    return result

def test_multiple_poses_honest(targets_pos, targets_ori, known_joints_list,
                               initial_guess_strategy='zero',
                               max_iterations=5000, tolerance=1e-6,
                               verbose=True):
    """
    Test IK for multiple poses with HONEST initial guesses that don't cheat.
    
    Args:
        targets_pos: Array of target positions [[x, y, z], ...]
        targets_ori: Array of target orientations [[rx, ry, rz], ...]
        known_joints_list: Array of known joint angles for each pose
        initial_guess_strategy: How to generate initial guesses:
            - 'zero': All joints at [0, 0, 0, 0, 0, 0]
            - 'random': Random angles within reasonable bounds
            - 'home': Common home position [0, -90, 90, 0, 0, 0]
            - 'constant': Use [10, 10, 10, 10, 10, 10] for all
        max_iterations: Max optimizer iterations
        tolerance: Convergence tolerance
        verbose: Print progress information
        
    Returns:
        pandas.DataFrame: Complete test results
    """
    results = []
    
    for i, (pos, ori, known) in enumerate(zip(targets_pos, targets_ori, known_joints_list)):
        if verbose and i % 10 == 0:
            print(f"Testing pose {i+1}/{len(targets_pos)}...")
        
        # Generate initial guess based on strategy
        if initial_guess_strategy == 'zero':
            q_init = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
        elif initial_guess_strategy == 'random':
            # Random angles between -90 and 90 degrees
            q_init = np.random.uniform(-90, 90, 6)
        elif initial_guess_strategy == 'home':
            q_init = np.array([0.0, -90.0, 90.0, 0.0, 0.0, 0.0])
        elif initial_guess_strategy == 'constant':
            q_init = np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0])
        else:
            raise ValueError(f"Unknown strategy: {initial_guess_strategy}")
        
        result = test_single_pose(
            pos, ori, known, q_init,
            max_iterations=max_iterations,
            tolerance=tolerance,
            test_index=i
        )
        
        results.append(result)
    
    return pd.DataFrame(results)

def test_multiple_poses_with_previous(targets_pos, targets_ori, known_joints_list,
                                     q_init=None, max_iterations=5000, tolerance=1e-6,
                                     verbose=True):
    """
    Test IK using previous solution as initial guess (EASY MODE - may hide problems).
    
    This is what your original code was doing - it makes IK look better than it is!
    """
    if q_init is None:
        q_init = known_joints_list[0]  # Start with first known joints
    
    results = []
    
    for i, (pos, ori, known) in enumerate(zip(targets_pos, targets_ori, known_joints_list)):
        if verbose and i % 10 == 0:
            print(f"Testing pose {i+1}/{len(targets_pos)}...")
        
        result = test_single_pose(
            pos, ori, known, q_init,
            max_iterations=max_iterations,
            tolerance=tolerance,
            test_index=i
        )
        
        results.append(result)
        
        # Use previous solution as next initial guess (THIS MAKES IT EASY!)
        if result['converged']:
            q_init = np.array([result[f'solved_j{j}'] for j in range(1, 7)])
    
    return pd.DataFrame(results)

def print_summary(results_df, strategy_name="Unknown"):
    """Print a clean, actionable summary of test results."""
    if len(results_df) == 0:
        print("No test results available.")
        return
    
    print("\n" + "="*80)
    print(f"INVERSE KINEMATICS TEST SUMMARY - Strategy: {strategy_name}")
    print("="*80)
    
    # Overall statistics
    total_tests = len(results_df)
    converged = results_df['converged'].sum()
    failed = total_tests - converged
    
    print(f"\n📊 Overall Performance:")
    print(f"   Total tests:      {total_tests}")
    print(f"   Converged:        {converged} ({100*converged/total_tests:.1f}%)")
    print(f"   Failed:           {failed} ({100*failed/total_tests:.1f}%)")
    
    if converged > 0:
        converged_df = results_df[results_df['converged']]
        
        # Quality distribution
        print(f"\n⭐ Quality Distribution (converged cases):")
        quality_counts = converged_df['quality'].value_counts()
        for quality in ['excellent', 'good', 'acceptable', 'poor', 'failed']:
            count = quality_counts.get(quality, 0)
            if count > 0:
                print(f"   {quality.capitalize():12s}: {count:4d} ({100*count/converged:.1f}%)")
        
        # Task space errors (what really matters!)
        print(f"\n🎯 Task Space Errors (using IK solution with FK):")
        print(f"   Position Error (mm):")
        print(f"      Mean:  {converged_df['fk_pos_error_norm'].mean():8.3f}")
        print(f"      Median:{converged_df['fk_pos_error_norm'].median():8.3f}")
        print(f"      Max:   {converged_df['fk_pos_error_norm'].max():8.3f}")
        print(f"      Min:   {converged_df['fk_pos_error_norm'].min():8.3f}")
        print(f"      Std:   {converged_df['fk_pos_error_norm'].std():8.3f}")
        
        # Per-coordinate errors
        print(f"\n   Position Error by Coordinate (mm):")
        for coord in ['x', 'y', 'z']:
            col = f'fk_pos_error_{coord}'
            mean_err = converged_df[col].abs().mean()
            max_err = converged_df[col].abs().max()
            print(f"      {coord.upper()}: mean={mean_err:7.3f}, max={max_err:7.3f}")
        
        # Joint space errors
        print(f"\n🔧 Joint Space Errors (degrees):")
        print(f"   Mean per-joint error: {converged_df['joint_error_mean'].mean():.3f}°")
        print(f"   Max per-joint error:  {converged_df['joint_error_max'].mean():.3f}°")
        print(f"   RMS per-joint error:  {converged_df['joint_error_rms'].mean():.3f}°")
        
        # Individual joint analysis
        print(f"\n   Error by Joint (degrees, absolute):")
        for i in range(1, 7):
            col = f'j{i}_error_abs'
            mean_err = converged_df[col].mean()
            max_err = converged_df[col].max()
            print(f"      J{i}: mean={mean_err:6.2f}°, max={max_err:6.2f}°")
        
        # Ground truth FK validation
        print(f"\n✅ Ground Truth Validation:")
        mean_known_error = converged_df['known_fk_pos_error'].mean()
        print(f"   FK with known joints error: {mean_known_error:.6f} mm")
        if mean_known_error < 1.0:
            print(f"   → Forward kinematics is working correctly! ✓")
        else:
            print(f"   → WARNING: FK errors suggest DH parameter issues!")
        
        # Key insight
        print(f"\n💡 Key Insight:")
        mean_task_error = converged_df['fk_pos_error_norm'].mean()
        mean_joint_error = converged_df['joint_error_mean'].mean()
        
        if mean_task_error < 5.0:
            print(f"   ✓ Task space errors are acceptable ({mean_task_error:.2f} mm)")
            print(f"     Joint errors ({mean_joint_error:.2f}°) map to good end-effector accuracy")
        elif mean_task_error < 10.0:
            print(f"   ⚠ Task space errors are marginal ({mean_task_error:.2f} mm)")
            print(f"     Consider improving optimization or joint error tolerance")
        else:
            print(f"   ✗ Task space errors are too large ({mean_task_error:.2f} mm)")
            print(f"     Joint errors ({mean_joint_error:.2f}°) lead to poor end-effector accuracy")
            print(f"     Action: Review IK optimization strategy")
    
    print("\n" + "="*80)

def get_worst_cases(results_df, n=5, metric='fk_pos_error_norm'):
    """Get the worst performing test cases for debugging."""
    converged_df = results_df[results_df['converged']].copy()
    if len(converged_df) == 0:
        print("No converged cases to analyze.")
        return pd.DataFrame()
    
    worst = converged_df.nlargest(min(n, len(converged_df)), metric)
    
    print(f"\n🔍 Top {min(n, len(converged_df))} Worst Cases (by {metric}):")
    print("-" * 80)
    
    for idx, row in worst.iterrows():
        print(f"\nCase {row['index']}:")
        print(f"  Target: ({row['target_x']:.1f}, {row['target_y']:.1f}, {row['target_z']:.1f}) mm")
        print(f"  FK Position Error: {row['fk_pos_error_norm']:.3f} mm")
        print(f"  Joint Errors (deg): ", end="")
        for i in range(1, 7):
            print(f"J{i}={row[f'j{i}_error_abs']:.1f}° ", end="")
        print()
    
    return worst

# %% [markdown]
# # Run HONEST IK Tests
# 
# Test your IK with different initial guess strategies to see how robust it really is.

# %%
print("="*80)
print("COMPREHENSIVE IK TESTING - MULTIPLE STRATEGIES")
print("="*80)

# Test 1: Zero initial guess (hardest test)
print("\n\n🔥 TEST 1: Zero Initial Guess (All joints at 0°)")
print("This is the HARDEST test - your IK must find the solution from scratch!")
results_zero = test_multiple_poses_honest(
    known_position,
    known_orientation,
    known_joint_angles,
    initial_guess_strategy='zero',
    max_iterations=5000,
    tolerance=1e-6,
    verbose=True
)
print_summary(results_zero, "Zero Initial Guess")
worst_zero = get_worst_cases(results_zero, n=3)

# %% 
# Test 2: Constant initial guess
print("\n\n🔸 TEST 2: Constant Initial Guess (All joints at 10°)")
print("Tests if your IK can handle a consistent but non-zero starting point.")
results_constant = test_multiple_poses_honest(
    known_position,
    known_orientation,
    known_joint_angles,
    initial_guess_strategy='constant',
    max_iterations=5000,
    tolerance=1e-6,
    verbose=True
)
print_summary(results_constant, "Constant Initial Guess")
worst_constant = get_worst_cases(results_constant, n=3)

# %%
# Test 3: Using previous solution (EASY MODE - what your code was doing)
print("\n\n✨ TEST 3: Using Previous Solution (EASY MODE)")
print("This makes IK look better because each guess is already close to the answer!")
print("This is what your original code was doing - it hides problems!")
results_previous = test_multiple_poses_with_previous(
    known_position,
    known_orientation,
    known_joint_angles,
    q_init=None,  # Will use first known joints
    max_iterations=5000,
    tolerance=1e-6,
    verbose=True
)
print_summary(results_previous, "Previous Solution (Easy Mode)")
worst_previous = get_worst_cases(results_previous, n=3)

# %%
# Save all results
output_dir = Path("/Users/yusufdineh/Documents/GitHub/EE347/Lab3")
results_zero.to_csv(output_dir / "ik_results_zero_init.csv", index=False)
results_constant.to_csv(output_dir / "ik_results_constant_init.csv", index=False)
results_previous.to_csv(output_dir / "ik_results_previous_init.csv", index=False)

print("\n\n💾 Results saved to:")
print(f"   {output_dir / 'ik_results_zero_init.csv'}")
print(f"   {output_dir / 'ik_results_constant_init.csv'}")
print(f"   {output_dir / 'ik_results_previous_init.csv'}")

# %%
# Compare strategies
print("\n\n" + "="*80)
print("STRATEGY COMPARISON")
print("="*80)

strategies = [
    ("Zero Init (Hard)", results_zero),
    ("Constant Init", results_constant),
    ("Previous Solution (Easy)", results_previous)
]

print(f"\n{'Strategy':<30} {'Success Rate':<15} {'Mean Error (mm)':<20} {'Quality':<15}")
print("-"*80)

for name, df in strategies:
    converged = df['converged'].sum()
    total = len(df)
    success_rate = f"{converged}/{total} ({100*converged/total:.1f}%)"
    
    if converged > 0:
        converged_df = df[df['converged']]
        mean_error = f"{converged_df['fk_pos_error_norm'].mean():.3f}"
        
        # Count excellent cases
        excellent = (converged_df['quality'] == 'excellent').sum()
        quality = f"{excellent}/{converged} excellent"
    else:
        mean_error = "N/A"
        quality = "N/A"
    
    print(f"{name:<30} {success_rate:<15} {mean_error:<20} {quality:<15}")

print("\n💡 If 'Previous Solution' works well but 'Zero Init' fails badly,")
print("   your IK algorithm is NOT robust - it only works when given good guesses!")

COMPREHENSIVE IK TESTING - MULTIPLE STRATEGIES


🔥 TEST 1: Zero Initial Guess (All joints at 0°)
This is the HARDEST test - your IK must find the solution from scratch!
Testing pose 1/14...
Testing pose 11/14...

INVERSE KINEMATICS TEST SUMMARY - Strategy: Zero Initial Guess

📊 Overall Performance:
   Total tests:      14
   Converged:        14 (100.0%)
   Failed:           0 (0.0%)

⭐ Quality Distribution (converged cases):
   Excellent   :   14 (100.0%)

🎯 Task Space Errors (using IK solution with FK):
   Position Error (mm):
      Mean:     0.015
      Median:   0.000
      Max:      0.051
      Min:      0.000
      Std:      0.021

   Position Error by Coordinate (mm):
      X: mean=  0.006, max=  0.041
      Y: mean=  0.010, max=  0.050
      Z: mean=  0.003, max=  0.012

🔧 Joint Space Errors (degrees):
   Mean per-joint error: 53.089°
   Max per-joint error:  176.325°
   RMS per-joint error:  91.490°

   Error by Joint (degrees, absolute):
      J1: mean=  1.00°, max=  5.32°
  

# Run Comprehensive IK Testing - Using Your Functions

This cell runs the complete test suite using **your own functions**:
- Uses your `fk()` function for forward kinematics
- Uses your `inverse_kinematics()` function for IK solving
- Uses your `link_lengths()` function for reach calculation

Generates:
- Detailed console output with statistics
- CSV file with all results
- Worst case analysis for debugging

In [12]:
# Run tests on all poses from your CSV data using your own functions
print("Starting comprehensive IK testing...")
print("This will test each pose and compute actual end-effector errors.\n")

results_df = test_multiple_poses(
    targets_pos=known_position,
    targets_ori=known_orientation,
    known_joints_list=known_joint_angles,
    q_init=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1],  # Will use first known joints as initial guess
    max_iterations=5000,
    tolerance=1e-6,
    use_previous_solution=True,  # Use previous solution as next initial guess
    verbose=True
)

# Print comprehensive summary
print_summary(results_df)

# Show worst 5 cases for debugging
worst_cases = get_worst_cases(results_df, n=5)

# Save results to CSV
output_path = Path("/Users/yusufdineh/Documents/GitHub/EE347/Lab3") / "ik_results_your_functions.csv"
save_results(results_df, output_path)

# Display first few rows of results
print("\n📋 Sample Results (first 5 rows):")
display(results_df.head())
# Analyze Specific Columns - Functional Version
results_df 

Starting comprehensive IK testing...
This will test each pose and compute actual end-effector errors.



NameError: name 'test_multiple_poses' is not defined

The most important columns in the CSV are:
- `fk_pos_error_norm`: **CRITICAL** - Actual end-effector position error when using IK solution
- `j1_error_abs` through `j6_error_abs`: Per-joint errors in degrees
- `quality`: Overall quality rating (excellent/good/acceptable/poor/failed)
- `converged`: Whether IK optimization succeeded

In [ ]:
# Quick analysis of key metrics using functional approach
if results_df['converged'].sum() > 0:
    converged_results = results_df[results_df['converged']]
    
    print("Key Metrics Summary:")
    print(f"  Avg Task Space Error: {converged_results['fk_pos_error_norm'].mean():.3f} mm")
    print(f"  Avg Joint Error: {converged_results['joint_error_mean'].mean():.3f}°")
    print(f"\nQuality Breakdown:")
    print(converged_results['quality'].value_counts())

Key Metrics Summary:
  Avg Task Space Error: 0.000 mm
  Avg Joint Error: 60.725°

Quality Breakdown:
quality
excellent    14
Name: count, dtype: int64
